In [11]:
import puffin.tardis_importer as ti
import numpy as np
import pandas as pd
import puffin
exchange_spot_tobs_file = puffin.config.source_directory + 'data/inputs/ftx_quotes_2021-10-14_BTC-USD.csv.gz'
exchange_spot_tobs = ti.generate_tob_data_from_gzip('BTC/USD', exchange_spot_tobs_file)
exchange_future_tobs_file = puffin.config.source_directory + 'data/inputs/ftx_quotes_2021-10-14_BTC-PERP.csv.gz'
exchange_future_tobs = ti.generate_tob_data_from_gzip('BTC/USD', exchange_future_tobs_file)
current_time = 1634176802000000000
latency = 0
spot_minimum_basis_spread_buffer = exchange_spot_tobs[exchange_spot_tobs['exchange_timestamp_nanos'].between(current_time - latency -60 * 60 * 1000000000,current_time - latency)]
future_minimum_basis_spread_buffer = exchange_future_tobs[exchange_future_tobs['exchange_timestamp_nanos'].between(current_time - latency -60 * 60 * 1000000000,current_time - latency)]
spot_minimum_basis_spread_buffer = spot_minimum_basis_spread_buffer[['exchange_timestamp_nanos','bid_price']]
future_minimum_basis_spread_buffer = future_minimum_basis_spread_buffer[['exchange_timestamp_nanos', 'ask_price']]

In [13]:
spread = pd.merge(spot_minimum_basis_spread_buffer, future_minimum_basis_spread_buffer, how='outer',left_on='exchange_timestamp_nanos', right_on='exchange_timestamp_nanos')
spread

,exchange_timestamp_nanos,bid_price,ask_price
0,1634173202081272000,57729.0,NaN
1,1634173202113122000,57729.0,NaN
2,1634173202201689000,57738.0,NaN
3,1634173202346518000,57742.0,NaN
4,1634173202434618000,57742.0,NaN
...,...,...,...
95799,1634176801810510000,NaN,58351.0
95800,1634176801834004000,NaN,58351.0
95801,1634176801860175000,NaN,58351.0
95802,1634176801912405000,NaN,58351.0


In [16]:
spread = spread.sort_values(by=['exchange_timestamp_nanos'])
spread.head(20)

,exchange_timestamp_nanos,bid_price,ask_price
35498,1634173202049985000,NaN,57772.0
0,1634173202081272000,57729.0,NaN
1,1634173202113122000,57729.0,NaN
35499,1634173202137132000,NaN,57772.0
35500,1634173202175484000,NaN,57772.0
2,1634173202201689000,57738.0,NaN
35501,1634173202253200000,NaN,57772.0
35502,1634173202300357000,NaN,57772.0
3,1634173202346518000,57742.0,NaN
35503,1634173202348557000,NaN,57772.0


In [17]:
spread = spread.fillna(method='ffill')
spread.head(20)

,exchange_timestamp_nanos,bid_price,ask_price
35498,1634173202049985000,NaN,57772.0
0,1634173202081272000,57729.0,57772.0
1,1634173202113122000,57729.0,57772.0
35499,1634173202137132000,57729.0,57772.0
35500,1634173202175484000,57729.0,57772.0
2,1634173202201689000,57738.0,57772.0
35501,1634173202253200000,57738.0,57772.0
35502,1634173202300357000,57738.0,57772.0
3,1634173202346518000,57742.0,57772.0
35503,1634173202348557000,57742.0,57772.0


In [18]:
spread = spread.dropna(how='any')
spread.head(20)

,exchange_timestamp_nanos,bid_price,ask_price
0,1634173202081272000,57729.0,57772.0
1,1634173202113122000,57729.0,57772.0
35499,1634173202137132000,57729.0,57772.0
35500,1634173202175484000,57729.0,57772.0
2,1634173202201689000,57738.0,57772.0
35501,1634173202253200000,57738.0,57772.0
35502,1634173202300357000,57738.0,57772.0
3,1634173202346518000,57742.0,57772.0
35503,1634173202348557000,57742.0,57772.0
35504,1634173202369956000,57742.0,57772.0


In [19]:
spread['spread'] = 2 * (spread['ask_price'] - spread['bid_price']) / (spread['ask_price'] + spread['bid_price'])
spread.head(20)

C:\Users\Tibor\AppData\Local\Temp/ipykernel_12092/3784679202.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spread['spread'] = 2 * (spread['ask_price'] - spread['bid_price']) / (spread['ask_price'] + spread['bid_price'])


,exchange_timestamp_nanos,bid_price,ask_price,spread
0,1634173202081272000,57729.0,57772.0,0.000745
1,1634173202113122000,57729.0,57772.0,0.000745
35499,1634173202137132000,57729.0,57772.0,0.000745
35500,1634173202175484000,57729.0,57772.0,0.000745
2,1634173202201689000,57738.0,57772.0,0.000589
35501,1634173202253200000,57738.0,57772.0,0.000589
35502,1634173202300357000,57738.0,57772.0,0.000589
3,1634173202346518000,57742.0,57772.0,0.000519
35503,1634173202348557000,57742.0,57772.0,0.000519
35504,1634173202369956000,57742.0,57772.0,0.000519


In [21]:
spread['spread'].median()

0.0006024563004019244

In [28]:
current_time = 1634185981000000000
spot_minimum_basis_spread_buffer = exchange_spot_tobs[exchange_spot_tobs['exchange_timestamp_nanos'].between(current_time - latency -60 * 60 * 1000000000,current_time - latency)]
future_minimum_basis_spread_buffer = exchange_future_tobs[exchange_future_tobs['exchange_timestamp_nanos'].between(current_time - latency -60 * 60 * 1000000000,current_time - latency)]
spot_minimum_basis_spread_buffer = spot_minimum_basis_spread_buffer[['exchange_timestamp_nanos','bid_price']]
future_minimum_basis_spread_buffer = future_minimum_basis_spread_buffer[['exchange_timestamp_nanos', 'ask_price']]
spread = pd.merge(spot_minimum_basis_spread_buffer, future_minimum_basis_spread_buffer, how='outer',left_on='exchange_timestamp_nanos', right_on='exchange_timestamp_nanos')
spread = spread.sort_values(by=['exchange_timestamp_nanos'])
spread = spread.fillna(method='ffill')
spread = spread.dropna(how='any')
spread['spread'] = 2 * (spread['ask_price'] - spread['bid_price']) / (spread['ask_price'] + spread['bid_price'])
spread['spread'].median()

0.000790364426728063